In [3]:
#!/usr/bin/python

import tweepy
import MySQLdb as mdb
import sys
import re
import urllib
import string
import sys
import pandas


#-----------Functions Defined -----------------------------------#


#-------------Getting DB Connection Function - Returns Connection --------#
def get_db_connection(config):
    #Creating connection to MYSQL DB
    try:
        con = mdb.connect(host = config["host"], 
                      user = config["user"], 
                      passwd = config["passwd"], 
                      db = config["db"], 
                      charset='utf8', use_unicode=True);
    except :
            print("Oops!  DB Connection Error!!!")

    return con

#--------------Get Cursor for seleted DB ---------------------#

def get_cursor(con):
    try:
    #DB  Name defined

        cur = con.cursor(mdb.cursors.DictCursor)
    except :
            print("Oops!  Cursor Fetch Error!!!")
            
    return cur

def close_db_connection(con):
    try:
    #DB  Name defined

        con.close()
    except :
            print("Oops!  Cannot close connection!!!")

            
            
            
def load_posneg_words(config,con):

    bool = True
    
    
    try:

        checkposnegexistquery = config["checkposnegexistquery"]

        cur.execute(checkposnegexistquery)    

        result =  cur.fetchone()

        number_of_rows=result['count(*)']
        
        #print number_of_rows
        if number_of_rows <= 0:

            #import negative and positive words

            #import negative words
            url = config["urlneg"]
            #url='http://www.unc.edu/~ncaren/haphazard/negative.txt'
            file_name=config["negativewrdsfn"]
            urllib.urlretrieve(url,file_name)

            #import positive words
            url = config["urlpos"]
            url='http://www.unc.edu/~ncaren/haphazard/positive.txt'
            file_name=config["positivewrdsfn"]
            urllib.urlretrieve(url,file_name)

            f = open(config["positivewrdsfn"], 'r')
            typeword = 'P'
            for line in f:
                #print line
                query_parameters = [line.strip()]
                cur.execute(config["selectwordsquery"],query_parameters)
                 #if tweet does not exist insert tweet alng with tweet in the table TweetsList
                qryresult =  cur.fetchone()

                number_of_rows=qryresult['count(*)']
        
                
                if number_of_rows <= 0:
                    query_parameters = [line.strip(),typeword]
                    cur.execute(config["insertwordsquery"], query_parameters)  
            f.close
            f = open(config["negativewrdsfn"], 'r')
            typeword = 'N'
            for line in f:
                #print line
                query_parameters = [line.strip()]
                cur.execute(config["selectwordsquery"],query_parameters)
                 #if tweet does not exist insert tweet alng with tweet in the table TweetsLi
                qryresult =  cur.fetchone()

                number_of_rows=qryresult['count(*)']
        
                
                if number_of_rows <= 0:                               
                    query_parameters = [line.strip(),typeword]
                    cur.execute(config["insertwordsquery"], query_parameters)  
            f.close        
            con.commit()   

    except:
        print("Oops!  Issue loading Positive/Negative words!!!")
        bool = False
    finally:
        cur.close()
        con.close()
    
    return bool


#-------------End of Functions ---------------------------------#        


#-------- Start of Main Program -----------------#
config = {}
execfile("config_1.txt", config)
subjects = ["Hillary","Hillary Clinton","Trump","Donald Trump"]
config["db"]

    
try:
#DB  Name defined

    con = get_db_connection(config)
    cur = get_cursor(con)
    #cur = con.cursor(mdb.cursors.DictCursor)
except :
        print("Oops!  Cursor Fetch Error!!!")

try:
    success =  load_posneg_words(config,con)
except :
    print("Oops!  PosNeg Fetch Error!!!")

for subject in subjects:

    #-----------------------------------------------------------------------
    # load our API credentials 
    #-----------------------------------------------------------------------



    #Authentication code for Twitter after reading config file
    auth = tweepy.OAuthHandler(config["consumer_key"], config["consumer_secret"])
    auth.secure = True
    auth.set_access_token(config["access_key"], config["access_secret"])
    api = tweepy.API(auth)



    #-----------------------------------------------------------------------
    # Twitter API Search Resuls 
    #-----------------------------------------------------------------------

    # Search Twitter for Tweets about two candidates
    
    results =  api.search(q=subject,rpp="1", show_user="true")
    if "Hillary" in subject:
        indicator = "H"
    if "Trump" in subject:
        indicator = "T"
            
    #Count defined for tweets recd and looping construct
    cnt = 0

    try:
    #DB  Name defined

        con = get_db_connection(config)
        cur = get_cursor(con)
        #cur = con.cursor(mdb.cursors.DictCursor)
    except :
            print("Oops!  Cursor Fetch Error!!!")
    #print indicator
    #-----------------------------------------------------------------------
    # loop through each of the tweets, get the user information who tweeted and store them in the DB
    #-----------------------------------------------------------------------
    for result in results:

        #Code to insert User who tweeted into DB for corresponding Tweet

        #getting the screen name and user id
        screenname = result.author.screen_name
        # api used to get userid based on screen name
        userinfo =  api.get_user(screen_name= screenname)
        userid = userinfo.id
        
        
        #Checking if the userid already exist in table tweetsuser
        query_parameters = [userid]    

        #if tweet does not exist insert userid along with screename in the table tweetsuser
        cur.execute(config["selectuserquery"],query_parameters)
        qryresult =  cur.fetchone()
        number_of_rows=qryresult['count(*)']

        if number_of_rows <= 0:
            query_parameters = (userid, userinfo.name,screenname,userinfo.location)
            cur.execute(config["inserttweetuserquery"], query_parameters)    
            
        #Checking if the tweetid already exist in table
        query_parameters = [result.id]    
        cur.execute(config["selecttweetquery"],query_parameters)
        qryresult =  cur.fetchone()
        number_of_rows=qryresult['count(*)']


        #if tweet does not exist insert tweet alng with tweet in the table TweetsList
        #if cur.rowcount <= 0:
        if number_of_rows <= 0:
            #print result.text
            number_of_rows = 0
            p = re.compile('(^RT @)(\w*):')    
            substr = p.split(result.text)
            tweettext = substr[len(substr)-1]
            query_parameters = (result.id, tweettext,indicator,userid)
            cur.execute(config["inserttweettextquery"], query_parameters)      
      
        

        #------------------------------------------------------
        #
        #-----------------------------------------------------
        positive_counter = 0
        negative_counter = 0
        tweet_processed=tweettext.lower()
        #print tweet_processed
        for p in list(string.punctuation):

            tweet_processed=tweet_processed.replace(p,'')
            words=tweet_processed.split(' ')

        for word in words:
            
            query_parameters = [word.strip()]   
            cur.execute(config["tweethasposnegquery"],query_parameters)

            #if tweet does not exist insert tweet alng with tweet in the table TweetsList
            qryresult =  cur.fetchone()
            number_of_rows=qryresult['count(*)']
            
            if number_of_rows > 0:
                typeword = qryresult["type"]  
                wordid = qryresult["wordid"]  
                
                query_parameters = [wordid,result.id]    
                cur.execute(config["inserttweetposnegwordquery"],query_parameters)
                
                if typeword == "P":
                    positive_counter=positive_counter+1
                if typeword == "N":
                    negative_counter = negative_counter+1
        



    con.commit()   
    cur.close()    
    con.close()




      


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:248: Warning: Invalid utf8 character string: 'F09F87'
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:248: Warning: Incorrect string value: '\xF0\x9F\x87\xBA\xF0\x9F...' for column 'tweetstext' at row 1
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:278: Warning: Invalid utf8 character string: 'F09F87'
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:278: Warning: Invalid utf8 character string: 'F09F92'
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:248: Warning: Invalid utf8 character string: 'F09F91'
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:248: Warning: Incorrect string value: '\xF0\x9F\x91\x8E\xF0\x9F...' for column 'tweetstext' at row 1
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:278: Warning: Invalid utf8 character string: 'F09F91'
